# I. California Housing Exploratory Data Analysis
### Prediction of median house prices for California districts derived from the 1990 census.

For this analysis we use the <b>California Housing</b> dataset found on the Kaggle Data
Repository at the following location:

<a href=https://www.kaggle.com/camnugent/california-housing-prices>California Housing Prices</a>

The objective of the analysis is to predict median house prices in different regions of California as per the 1990
census data.

We hope to gain valuable insights by following this process. The various steps in the
process can be elaborated on as follows:

- Load data
- Prepare data
    - Clean data
        - Missing values
        - Outliers
        - Erroneous values
    - Explore data
        - Exploratory descriptive analysis (EDA)
        - Correlation analysis
        - Variable cluster analysis
    - Transform Data
        - Engineer features
        - Encode data
        - Scale & normalise data
        - Impute data (if not done in previous steps)
        - Feature selection/ importance analysis
- Build model
    - Model selection
    - Data sampling (validation strategy, imbalanced classification)
    - Hyperparameter optimisation
- Validate model
    - Accuracy testing
- Analysis of results
    - Response curves
    - Accuracy analysis
    - Commentary

This dataset is used in the second chapter of Aurélien Géron's recent book 'Hands-On Machine learning with
Scikit-Learn and TensorFlow' - highly recommended!

The data dictionary for this dataset is as follows:

| Variable | Definition | Type |
|----------|------------|-----|
| longitude| Longitude	| Continuous |
| latitude | Latitude   | Continuous |
| housingmedianage | Median age of houses in area | Continuous |
| total_rooms | Avg No. of rooms of houses in area | Continuous |
| total_bedrooms | Avg No. of bedrooms of houses in area | Continuous |
| population | No. of people in area | Continuous |
| households | No. of households in area | Continuous |
| median_income | Median income of household | Continuous |
| medianhousevalue | Median value of houses in area | Continuous |
| ocean_proximity | Proximity to ocean	| Categorical |

Let us start the exploration!

In [ ]:
# Import libraries
from subprocess import call

import patsy
import folium as folium
from folium import plugins
from folium.plugins import HeatMap
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.core.display import Image
from matplotlib import pyplot
from numpy import isnan
from patsy.highlevel import dmatrices, dmatrix
from sklearn import model_selection
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2
from sklearn.impute import KNNImputer
from sklearn.impute._iterative import IterativeImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.model_selection import GridSearchCV
from pandas.plotting import scatter_matrix
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

<div class="alert alert-block alert-info">
<b>Load data</b>
</div>

In [ ]:
# Import data
df_prices = pd.read_csv('../input/california-housing-prices/housing.csv', delimiter = ",")
df_orig = df_prices.copy()
df = df_prices.copy()


#### Entire set features

In [ ]:
df_prices.head(20)
print(df_prices.shape)

We will peform our analysis as follows:

- Missing data checks.
- Model accuracy check.
- EDA (without missing data replacement).
- Variable adjustments & missing value imputation.
- Model accuracy check

<div class="alert alert-block alert-info">
<b>Missing values</b>
</div>

We quantify the number of missing values in the data set:

In [ ]:
# Check for null values
missing_values_train = df_prices.isnull().sum()
missing_values_train = missing_values_train.to_frame(name='num_missing')
missing_values_train['perc_missing'] = (missing_values_train['num_missing']/df_prices.shape[0])*100
for index, row in missing_values_train.iterrows():
    if (row['num_missing'] > 0):
        print ("For \"%s\" the number of missing values are: %d (%.0f%%)" %  (index,
                                                                     row['num_missing'],
                                                                    row['perc_missing']))

Consider a sample of missing values from the data set:

In [ ]:
df_prices[df_prices.isnull().any(axis=1)]

We observed 207 (1%) null values for <i>total_bedrooms</i>.

The visualised missing values look as follows for the training set:

In [ ]:
#%matplotlib inline
_ = plt.figure(figsize=(20, 10))

# cubehelix palette is a part of seaborn that produces a colormap
cmap = sns.cubehelix_palette(light=1, as_cmap=True, reverse=True)
_ = sns.heatmap(df_prices.isnull(), cmap=cmap)

We will start by analysing the distributions for median house value first to get a feel for the data.

In [ ]:
# Continuous density plot
fig_missing, axes = plt.subplots(1, 1, figsize=(10, 8))

# Plot frequency plot/ histogram
_ = sns.histplot(x="median_house_value", kde=True, data=df_prices, ax=axes, bins=40);
_ = axes.set(xlabel="House Value", ylabel='Density');
axes.xaxis.label.set_size(18)
axes.yaxis.label.set_size(18)
axes.tick_params('y', labelsize = 14);
axes.tick_params('x', labelsize = 14);

In [ ]:
print("House value summary statistics:\n")
print(df_prices['median_house_value'].describe())

There is a skew to the right with a huge spike at high end of prices - this is likely to be a data issue. We will
have to investigate.
Maximum price was 500 001 and minimum age was around 14 999 dollars.

In [ ]:
# Replace missing values
df_train = df_prices.copy()
median = df_train['total_bedrooms'].median()
df_train['total_bedrooms'].fillna(median, inplace=True)
print("Number of null values in total bedrooms column: {}".format(df_train['total_bedrooms'].isnull().sum()))

We observe that the null values have been removed. We now have a dataset ready for further analysis -
albeit a bit of a black box hack :) We will now do some very limited EDA just to get a feel for the data
as previously discussed.

<div class="alert alert-block alert-info">
<b>Exploration of data</b>
</div>

We start by looking at the number of unique records per variable.

In [ ]:
print(df_train.nunique())

There are no columns with only one value. We therefore retain all columns for ML purposes as there is
enough variability to warrant using the data.

We start by separating continuous and categorical variables for further high level analysis.

In [ ]:
all_vars = list(df_prices.columns.values)
print(all_vars)

In [ ]:
# Separate continuous and categorical variables
names_con = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population',
             'households', 'median_income', 'median_house_value']
names_cat = ['ocean_proximity']

df_train_con = df_train.loc[:, names_con]
df_train_cat = df_train.loc[:, names_cat]

### High level overview

Let's continue with the high level analysis.

#### Continuous variable overview
Most of the variables are continuous

In [ ]:
_ = df_prices.hist(bins=50, figsize=(20,15))
_ = plt.savefig("attribute_histogram_plots")
plt.show()

#### Categorical variable overview

In [ ]:
proximity_counts = df_train_cat['ocean_proximity'].value_counts(normalize = True)
legend_labels = ['<1H Ocean', 'Inland', 'Near Ocean', 'Near Bay', 'Island']

# change the background bar colors to be light grey
bars = plt.bar(proximity_counts.index, proximity_counts.values, align='center', linewidth=0,
               color='lightslategrey')
# make one bar, the <1H Ocean bar, a contrasting color
bars[0].set_color('#1F77B4')

# soften all labels by turning grey
_ = plt.xticks(proximity_counts.index, legend_labels, fontsize=11, alpha=0.8)
_ = plt.title('Ocean Proximity', fontsize=14, pad=30, alpha=0.8)

# remove all the ticks (both axes), and tick labels on the Y axis
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False,
                labelbottom=True)

# Remove the frame - my method
ax = plt.gca()
ax.set_frame_on(False)

Class percentages:

In [ ]:
# Class level counts for categorical variables.
for variable in names_cat:
    print(df_prices[variable].value_counts())
    print("\n")

We now have a look at correlation in the dataset. We use Pearson's Correlation as most of the variables are continuous.

In [ ]:
fig = plt.figure();
_ = scatter_matrix(df, figsize = (25,25), alpha=0.9, diagonal="kde", marker="o");

Let's have a look at a heatmap of prices.

In [ ]:
california_map = folium.Map(location=[36.7783,-119.4179], zoom_start = 6, min_zoom=5)
df_map = df_prices[['latitude', 'longitude']]
data = [[row['latitude'],row['longitude']] for index, row in df_map.iterrows()]
_ = HeatMap(data, radius=10).add_to(california_map)
california_map

Now let us try to find the areas where prices are highest by using a Decision Tree.

In [ ]:
X = df_prices.loc[:,["longitude","latitude"]]
y = df_prices.loc[:,"median_house_value"]

In [ ]:
regressor = DecisionTreeRegressor(random_state=0)
regressor = regressor.fit(X,y)

In [ ]:
# Visualising the decision tree regression results
_ = plt.figure(figsize=(10,10), dpi=150)
_ = plot_tree(regressor,max_depth=2,feature_names=X.columns, impurity=False, filled=True)
_ = plt.show()

Let us do a simple scatter plot of longitude vs latitude to see shape of price distribution.

In [ ]:
# A scatter plot of latitude vs longitude
_ = df.plot.scatter(x='longitude',y='latitude',c='DarkBlue',s=1.5)

Let us now annotate this plot with the first two splits in the Decision Tree.


In [ ]:
_ = plt.figure(figsize=[6,4], dpi=120)
_ = plt.ylim(32,42)
_ = plt.xlim(-125,-114)
_ = plt.ylabel("Latitude")
_ = plt.xlabel("Longitude")
_ = plt.title("Median House Value")
# A scatter plot of latitude vs longitude
_ = plt.scatter(x=X["longitude"],y=X["latitude"],c=df["median_house_value"], s=1.5)
# Show first two decision points
splits = regressor.tree_.threshold[:2]
_ = plt.plot([-125,-114],[splits[0],splits[0]])
_ = plt.plot([splits[1],splits[1]],[32,splits[0]])
_ = plt.colorbar()
_ = plt.show()

To conclude this analysis, let's see just how predictive "location" is in terms of house price.

We will build a simple Decision Tree model based on location only and do an accuracy test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
regressor_acc = regressor.fit(X_train, y_train)
y_pred = regressor_acc.predict(X_test)
kfold = model_selection.KFold(n_splits=10, random_state=7, shuffle=True)

# MSE - take absolute value first
scoring = 'neg_mean_squared_error'
results = model_selection.cross_val_score(regressor_acc, X, y, cv=kfold, scoring=scoring)
print("MSE: %.3f (%.3f)" % (results.mean(), results.std()))

# R-Squared
scoring = 'r2'
results = model_selection.cross_val_score(regressor_acc, X, y, cv=kfold, scoring=scoring)
print("R^2: %.3f (%.3f)" % (results.mean(), results.std()))

# Mean Absolute Error
scoring = 'neg_mean_absolute_error'
results = model_selection.cross_val_score(regressor_acc, X, y, cv=kfold, scoring=scoring)
print("MAE: %.3f (%.3f)" % (results.mean(), results.std()))

Based on this simple EDA and model accuracy results we can confirm that location is a very strong predictor of house
prices indeed. An R-squared of 73% is a very strong indication of strong correlation between location and house price
. This is however not a surprise as everyone know that it is all about: "location, location, location"!

In our next notebook we will do some more EDA and investigate correlation patterns with other variables. We will then
 build some predictive models using Pipelines. Cannot wait!

